<a href="https://colab.research.google.com/github/HRS5020/Tensorflow/blob/main/NLP_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary classifier with IMDB reviews dataset


In [1]:
# download the data , run this cell only once.

import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG0D7FS/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG0D7FS/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew

In [2]:
print(imdb)

{'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [16]:
for i,j in imdb['test'].take(2):
    print('Data = ', i,'\n')
    print('Label = ', j, '\n')

Data =  tf.Tensor(b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.", shape=(), dtype=string) 

Label =  tf.

In [23]:
# data preprocessing

import numpy as np

training_data = []
test_data = []
training_labels = []
test_labels = []

# store each review and label in an array
for i,j in imdb['train']:
    training_data.append(i.numpy().decode('utf8'))
    training_labels.append(j.numpy())

for i,j in imdb['test']:
    test_data.append(i.numpy().decode('utf8'))
    test_labels.append(j.numpy())


print(training_data[:5])
print(training_labels[:5])

# convert labels into numpy array
training_labels = np.array(training_labels)
test_labels = np.array(test_labels)

["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was

In [36]:
# Parameters

vocab_size = 10000
max_length = 120
embedding_dim = 64
trunc_type='post'
oov_tok = "<OOV>"

In [37]:
# generate padded sequences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_data)

word_index = tokenizer.word_index
# print(word_index)

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_data)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(test_data)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [40]:
# build the  model

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 120, 64)           640000    
                                                                 
 flatten_3 (Flatten)         (None, 7680)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               3932672   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               65664     
                                                                 
 dense_10 (Dense)            (None, 1)                 129       
                                                                 
Total params: 4,638,465
Trainable params: 4,638,465
No

In [41]:
num_epochs = 10

# Train the model
model.fit(padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, test_labels))


Epoch 1/10
782/782 [==============================] - 40s 50ms/step - loss: 0.4548 - accuracy: 0.7724 - val_loss: 0.3641 - val_accuracy: 0.8398
Epoch 2/10
782/782 [==============================] - 39s 50ms/step - loss: 0.1257 - accuracy: 0.9545 - val_loss: 0.5430 - val_accuracy: 0.8040
Epoch 3/10
782/782 [==============================] - 39s 50ms/step - loss: 0.0223 - accuracy: 0.9923 - val_loss: 0.8354 - val_accuracy: 0.8046
Epoch 4/10
782/782 [==============================] - 39s 50ms/step - loss: 0.0163 - accuracy: 0.9946 - val_loss: 0.8930 - val_accuracy: 0.8059
Epoch 5/10
782/782 [==============================] - 39s 50ms/step - loss: 0.0131 - accuracy: 0.9960 - val_loss: 1.0655 - val_accuracy: 0.8002
Epoch 6/10
782/782 [==============================] - 39s 50ms/step - loss: 0.0123 - accuracy: 0.9959 - val_loss: 1.1325 - val_accuracy: 0.8084
Epoch 7/10
782/782 [==============================] - 39s 50ms/step - loss: 0.0087 - accuracy: 0.9970 - val_loss: 1.1579 - val_accuracy: